### Testing/Submission Notebook - Histopathology Cancer Detection

In [ ]:
# Import necessary libraries
import torch
from scripts.data_utils import HistopathologyDataModule
from scripts.model_utils import BaselineCNN
from scripts.config import TEST_DIR, BATCH_SIZE, TARGET_SIZE

### Load Model and Test Data

In [ ]:
# Instantiate data module and load test dataset
data_module = HistopathologyDataModule(batch_size=BATCH_SIZE, target_size=TARGET_SIZE)
data_module.setup(stage="test")

### Generate Predictions

In [ ]:
# Load trained model checkpoint
model_path = "checkpoints/best_model.ckpt"
model = BaselineCNN.load_from_checkpoint(model_path)

In [ ]:
# Generate predictions and save them as submission.csv
model.generate_predictions(data_module.test_dataloader(), output_path="submission.csv")

In [ ]:
# Uncomment if running on Kaggle and sync back to GitHub
# !git add submission.csv
# !git commit -m "Add submission file for Kaggle"
# !git push origin main